# Set Up

## Import Packages

In [ ]:
# Lavorare con i dataframe
import pandas as pd
#pd.set_option('display.max_rows', None)

# Importare sys per indicare il path della cartella ove sono collocate le funzioni selfmade.
import sys
sys.path.append('/Users/mattia/Desktop/MatchAnalysis_Imputation/Progetto/Funzioni')
from funzioni import alfabeto, pulisci_prezzo # Funzioni fatte da se


path_data_lake = "/Users/mattia/Desktop/MatchAnalysis_Imputation/Progetto/Data Lake"
path_data_product = "/Users/mattia/Desktop/MatchAnalysis_Imputation/Progetto/Data Product"

## Load the Data

Importare i dataframe contenenti informazioni su hotel e B&B riguardanti le città di Roma e Terni disponibili
sulle piattaforme di agoda e booking.

In [17]:
agoda_terni = pd.read_csv(f'{path_data_lake}/agoda_Terni.csv',index_col=0)
booking_terni = pd.read_csv(f'{path_data_lake}/booking_Terni.csv',index_col=0)

booking_roma = pd.read_csv(f'{path_data_lake}/booking_Roma.csv',index_col=0)
agoda_roma = pd.read_csv(f'{path_data_lake}/agoda_Roma.csv',index_col=0)

# Data overview

Osservare i dati e capire se c'è possibilità di feature engineering, capire inoltre se ci sono delle variabili
che necessitano di pulizia.

## Dati riguardanti Agoda

Controllare che le variabili tra sul sito di **agoda** riguardo a Roma e Terni siano le stesse.

In [53]:
agoda_terni.head(1)

,titolo,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,prezzo,indirizzo,valutazione
0,Hotel Valentino,"9,0",Eccezionale,2 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",258 €,"Terni, Terni",4.0


In [ ]:
agoda_terni.columns.equals(agoda_roma.columns)

True

Le variabili tra agoda Roma e agoda Terni sono le stesse e sono:
   1. **Titolo** -> Nome della struttura;

   2. **Recensione voto numerico** -> Recensione con voto numerico;

   3. **Recensione voto parola** -> Recensione con voto a parola;

   4. **Numero recensioni** -> Numero di recensioni fatte alla struttura;

   5. **Date** -> Data di permanenza nella struttura;

   6. **Permanenza** -> Informazioni sulla permanenza, numero di notti e numero di personev(adulti o bambini);

   7. **Prezzo** -> Costo della permanenza;

   8. **Indirizzo** -> Indirizzo della struttura formato (città, provincia);

   9. **Valutazione** -> Le stelle di un hotel o la valutazione assegnata al B&B.

Sul dataset di agoda si può fare del feature engineering:

   - Creare variabile NUMERO NOTTI dalla variabile **permanenza**, deve essere un numero;

   - Creare variabile NUMERO PERSONE (adulte) e NUMERO PERSONE (bambini) dalla variabile **permanenza**;

   - Creare variabile DATA CHECK IN da **date**;

   - Creare variabile DATA CHECK OUT da **date**.



Per quanto riguarda la variabile **indirizzo**:
   - Nel dataframe **ROMA** ha il formato (zona, città - distanza centro)  es. Stazione centrale Roma Termini, Roma - In pieno centro;

   - Nel dataframe **TERNI** ha il formato (zona, città)  es. Terni, Terni;


Perciò per quanto riguarda Roma può anche essere creata la variabile **distanza dal centro**, oltre a **zona** e **città**.      
Invece riguarda Terni può essere creata la variabile **zona** e **città**.                  

Inoltre per effettuare le dovute analisi bisogna:
   - Ricodificare la variabile **prezzo** rimuovendo il simbolo dell'euro (accertarsi che i prezzi siano tutti in euro);

   - Sempre riguardo alla variabile **prezzo** eliminare il punto come separatore delle migliaia;

   - **numero recensioni** qualora comparisse "Novità su Agoda" inserire 0 (zero).

   - **numero recensioni** estrarre solo il numero di recensioni eliminado la stringa **recensioni** contenuta.

   - La variabile **distanza centro** andrà pulita e resa un numero (float).

## Dati riguardanti booking

Controllare che le variabili tra sul sito di **booking** riguardo a Roma e Terni siano le stesse.

In [ ]:
booking_roma.head(1)

,titolo,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,prezzo,distanza_centro,indirizzo,stelle,valutazione_booking,descrizione_camera,configurazione_camera,descrizione_unprocessed
0,The Sereno-3,"8,1",Ottimo,10 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€ 242,"3,5 km dal centro","Trionfale, Roma",NaN,4.0,Appartamento con 2 Camere da Letto e Vista Città,Intero appartamento • 2 camere da letto • 1 ba...,Appartamento con 2 Camere da Letto e Vista Cit...
1,Sonder by Marriott Bonvoy Piazza Venezia Apart...,"8,1",Ottimo,321 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€ 322,150 m dal centro,"Pantheon, Roma",3.0,NaN,Camera Queen,Configurazione camera non disponibile,Camera Queen 1 letto matrimonialeResta solo 1 ...
2,GuestHouse Martin,"8,8",Favoloso,267 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€ 69,"3,3 km dal centro","Villa Borghese/Parioli, Roma",NaN,4.0,Camera Doppia con Letti Singoli con Bagno Priv...,Configurazione camera non disponibile,Camera Doppia con Letti Singoli con Bagno Priv...
3,ASTORIA GOLDEN GATE,"8,9",Favoloso,1.446 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€ 110,2 km dal centro,"Stazione Termini, Roma",2.0,NaN,Camera Matrimoniale Superior,Configurazione camera non disponibile,Camera Matrimoniale Superior Vari tipi di lett...
4,Magnifico Rome,"7,8",Buono,781 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€ 107,"1,1 km dal centro","Stazione Termini, Roma",NaN,4.0,Camera Matrimoniale Economy,Configurazione camera non disponibile,Camera Matrimoniale Economy1 letto matrimonial...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
811,Boutique Hotel Campo de' Fiori,"8,8",Favoloso,805 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€ 499,"0,8 km dal centro","Piazza Navona, Roma",3.0,NaN,Camera Matrimoniale,Configurazione camera non disponibile,Camera Matrimoniale1 letto matrimonialeCancell...
812,Daphne Suites,"7,7",Buono,961 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€ 163,"0,9 km dal centro","Trevi, Roma",NaN,3.0,Camera Matrimoniale Comfort,Configurazione camera non disponibile,Camera Matrimoniale Comfort 1 letto matrimonia...
813,Tridente Suites,"9,3",Eccellente,913 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€ 185,"1,6 km dal centro","Spagna, Roma",NaN,4.0,Camera Matrimoniale/Doppia con Letti Singoli,Configurazione camera non disponibile,Camera Matrimoniale/Doppia con Letti SingoliVa...
814,AD2015 Guest House,"9,1",Eccellente,542 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€ 164,"2,5 km dal centro","Vaticano Prati, Roma",NaN,4.0,Double Room with St. Peter View,Configurazione camera non disponibile,Double Room with St. Peter ViewVari tipi di le...


In [38]:
booking_terni.columns.equals(booking_roma.columns)

True

Le variabili tra agoda Roma e agoda Terni sono le stesse e sono:
   1. **Titolo** -> Nome della struttura;

   2. **Recensione voto numerico** -> Recensione con voto numerico;

   3. **Recensione voto parola** -> Recensione con voto a parola;

   4. **Numero recensioni** -> Numero di recensioni fatte alla struttura;

   5. **Date** -> Data di permanenza nella struttura;

   6. **Permanenza** -> Informazioni sulla permanenza, numero di notti e numero di personev(adulti o bambini);

   7. **Prezzo** -> Costo della permanenza;

   8. **Distanza centro** -> Distanza dal centro della città di riferimento; 

   9. **Indirizzo** -> Indirizzo della struttura formato (città, provincia);

   10. **Stelle** -> Stelle se la struttura è un hotel (NAN se non è hotel);

   11. **Valutazione booking** -> Valutazione assegnata al B&B (NAN se non è B&B);

   12. **Descrizione camera** -> Descrizione della camera, quante camere se c'è vista ecc...;

   13. **Configurazione camera** -> Se c'è il bagno, numero di letti ecc...;

   14. **Descrizione unprocessed** -> Descrizione camera + Configurazione camera.

Sul dataset di booking si può fare del feature engineering:
   - Creare variabile NUMERO NOTTI dalla variabile **permanenza**, deve essere un numero;

   - Creare variabile NUMERO PERSONE (adulte) e NUMERO PERSONE (bambini) dalla variabile **permanenza**;

   - Creare variabile DATA CHECK IN da **date**;

   - Creare variabile DATA CHECK OUT da **date**.

Per quanto riguarda la variabile **indirizzo**:
   - Nel dataframe **ROMA** ha il formato (zona, città)  es. Trionfale, Roma;

   - Nel dataframe **TERNI** ha il formato (città)  es. Terni;


Perciò per quanto riguarda Roma può anche essere creata la variabile **distanza dal centro**, oltre a **zona** e **città**.      
Invece riguarda Terni può essere creata la variabile **zona** e **città**.   

Inoltre per effettuare le dovute analisi bisogna:
   - Ricodificare la variabile **prezzo** rimuovendo il simbolo dell'euro (accertarsi che i prezzi siano tutti in euro);

   - Sempre riguardo alla variabile **prezzo** eliminare il punto come separatore delle migliaia;

   - **numero recensioni** qualora comparisse "Novità su booking" inserire 0 (zero).

   - **numero recensioni** estrarre solo il numero di recensioni eliminado la stringa **recensioni** contenuta.

   - La variabile **distanza centro** andrà pulita e resa un numero (float).

# Data Cleaning

## Cleaning dei dati riguardanti Agoda

### Agoda Terni

### Agoda Roma

In [4]:
# Aggiungere variabili e modificarle (feature engeneering)
## Estrarre solo il primo nome, assegnarlo a variabile 'città'
agoda_terni['città'] = agoda_terni['indirizzo'].str.split(',').str[0].str.strip()
agoda_terni['comune'] = agoda_terni['indirizzo'].str.split(',').str[0].str.strip()

## Creare nuova variabile città, utile in seguito
booking_terni['città'] = booking_terni['indirizzo']

## Rinominare città Papigno come Terni
agoda_terni.loc[agoda_terni['città'] == 'Papigno', 'città'] = 'Terni'

## Creare variabile città
booking_roma['città']='Roma'
agoda_roma['città']='Roma'

# Facciamo merge dei due dataset separatamente per agoda e booking
## Unione dei dataset Agoda
agoda = pd.concat([agoda_terni, agoda_roma], ignore_index=True)

## Unione dei dataset Booking
booking = pd.concat([booking_terni, booking_roma], ignore_index=True)



For the purposes of our analysis, we only want to know a title. Let's toss the other columns so we have something more manageable to work with.

In [5]:
agoda["titolo"]=agoda.titolo.apply(lambda x: alfabeto(x))

booking["titolo"]=booking.titolo.apply(lambda x: alfabeto(x))

In [6]:
# Take a first look at the data
agoda.head()

,Unnamed: 0,titolo,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,prezzo,indirizzo,valutazione,città,comune
0,0,hotel valentino,"9,0",Eccezionale,2 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",258 €,"Terni, Terni",4.0,Terni,Terni
1,1,garden hotel terni,"7,9",Ottimo,505 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",138 €,"Terni, Terni",4.0,Terni,Terni
2,2,hotel michelangelo palace spa,"8,0",Fantastico,718 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",174 €,"Terni, Terni",4.0,Terni,Terni
3,3,classic hotel tulipano,"7,6",Ottimo,453 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",98 €,"Terni, Terni",3.0,Terni,Terni
4,4,de hotel paris,"8,1",Fantastico,11 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",69 €,"Terni, Terni",3.0,Terni,Terni


In [7]:
booking.head()

,Unnamed: 0,titolo,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,prezzo,distanza_centro,indirizzo,stelle,valutazione_booking,descrizione_camera,configurazione_camera,descrizione_unprocessed,città
0,0,de hotel paris,"7,2",Buono,869 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€ 69,"0,9 km dal centro",Terni,3.0,NaN,Camera Doppia con Letti Singoli,Configurazione camera non disponibile,Camera Doppia con Letti Singoli2 letti singoli...,Terni
1,1,bb cospea,"9,0",Eccellente,674 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€ 55,"1,7 km dal centro",Terni,NaN,3.0,Camera Doppia con Letti Singoli,Configurazione camera non disponibile,Camera Doppia con Letti Singoli2 letti singoli...,Terni
2,2,bedbreakfast rosy,"9,0",Eccellente,330 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€ 77,1 km dal centro,Terni,NaN,4.0,Camera Matrimoniale con Bagno Privato Esterno,Configurazione camera non disponibile,Camera Matrimoniale con Bagno Privato Esterno ...,Terni
3,3,angolo bb verde,"8,6",Favoloso,219 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€ 69,"2,6 km dal centro",Terni,NaN,3.0,Camera Matrimoniale,Configurazione camera non disponibile,Camera Matrimoniale1 letto matrimoniale alla f...,Terni
4,4,filomena guest house santa,"9,7",Eccezionale,11 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€ 74,1 km dal centro,Terni,NaN,3.0,Camera Tripla,Configurazione camera non disponibile,"Camera Tripla2 letti (1 singolo, 1 matrimonial...",Terni


In [8]:
# List of columns to keep
selected_columns = ['titolo','prezzo', 'indirizzo','città','numero_recensioni','valutazione_booking']

# Subset data frame to only include those columns
booking_reduced = booking[selected_columns]


In [9]:
# Look at the updated dataframe
booking_reduced.head()

,titolo,prezzo,indirizzo,città,numero_recensioni,valutazione_booking
0,de hotel paris,€ 69,Terni,Terni,869 recensioni,NaN
1,bb cospea,€ 55,Terni,Terni,674 recensioni,3.0
2,bedbreakfast rosy,€ 77,Terni,Terni,330 recensioni,4.0
3,angolo bb verde,€ 69,Terni,Terni,219 recensioni,3.0
4,filomena guest house santa,€ 74,Terni,Terni,11 recensioni,3.0


In [10]:
selected_columns = ['titolo','prezzo', 'indirizzo','città','numero_recensioni','valutazione']
agoda_reduced = agoda[selected_columns]
agoda_reduced.head()

,titolo,prezzo,indirizzo,città,numero_recensioni,valutazione
0,hotel valentino,258 €,"Terni, Terni",Terni,2 recensioni,4.0
1,garden hotel terni,138 €,"Terni, Terni",Terni,505 recensioni,4.0
2,hotel michelangelo palace spa,174 €,"Terni, Terni",Terni,718 recensioni,4.0
3,classic hotel tulipano,98 €,"Terni, Terni",Terni,453 recensioni,3.0
4,de hotel paris,69 €,"Terni, Terni",Terni,11 recensioni,3.0


#pulizia variabile indirizzo

In [11]:
#nuova pulizia indirizzo
agoda_reduced['indirizzo'] = (
    agoda_reduced['indirizzo']
    .str.lower()  # tutto minuscolo
    .str.replace(r'\s*-\s*.*$', '', regex=True)  # rimuove tutto dopo il trattino incluso
    .str.replace(r',?\s*roma\b', '', regex=True)  # rimuove "roma" opzionalmente preceduta da virgola
    .str.replace(r'\bcentrale\b', '', regex=True)  # rimuove "centrale" come parola intera
    .str.replace(r'\b(rione|piazza|fontana|di|del|e|città|citta|citt)\b', '', regex=True)  # rimuove parole specificate
    .str.replace(r'\s+', ' ', regex=True)  # rimuove spazi multipli eventualmente lasciati
    .str.strip()  # rimuove spazi iniziali/finali
)


/var/folders/j4/9fk3st2d30g22vff83pj6cnw0000gn/T/ipykernel_8489/3028220868.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agoda_reduced['indirizzo'] = (


In [12]:
agoda_reduced.indirizzo.unique()

array(['terni, terni', 'san gemini, san gemini', 'narni, narni',
       'stroncone, stroncone', 'montefranco, montefranco',
       "calvi dell' umbria, terni", 'papigno, papigno',
       'stazione termini', 'via veneto', 'trionfale', 'appio latino',
       'pantheon', 'eur garbatella', 'aurelio monteverde',
       'aeroporto fiumicino', 'ovest', 'nord', 'est', 'trastevere',
       'trevi', 'vaticano', 'flaminio parioli', 'boccea', 'nomentano',
       'navona', 'monti', 'colosseo foro romano', 'policlinico',
       'aeroporto ciampino', 'ostia', 'spagna', 'san giovanni'],
      dtype=object)

In [13]:
booking_reduced['indirizzo'] = (
    booking_reduced['indirizzo']
    .str.lower()  # tutto minuscolo
    .str.replace(r',?\s*roma\b', '', regex=True)  # rimuove "roma" opzionalmente preceduta da virgola
    .str.replace(r'\bcentrale\b', '', regex=True)  # rimuove "centrale" come parola intera
    .str.replace(r'\b(rione|piazza|fontana|di|del|e|città|citta|citt)\b', '', regex=True)  # rimuove parole specificate
    .str.replace(r'\s+', ' ', regex=True)  # rimuove spazi multipli eventualmente lasciati
    .str.strip())

/var/folders/j4/9fk3st2d30g22vff83pj6cnw0000gn/T/ipykernel_8489/542438600.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  booking_reduced['indirizzo'] = (


In [14]:
booking_reduced.indirizzo.unique()

array(['terni', 'trionfale', 'pantheon', 'villa borghese/parioli',
       'stazione termini', 'gianicolense', 'monti', '', 'centro',
       'san giovanni', 'navona', 'vaticano prati', 'trevi', 'via veneto',
       'spagna', 'trastevere', 'colosseo', 'aventino', 'tor quinto',
       'ostiense', 'eur', 'nomentano', 'magliana vecchia',
       'giuliano-dalmata', 'fonte ostiense', 'aurelio', 'appio latino',
       'portuense', 'torrino', 'monte sacro', 'san lorenzo', 'tiburtino',
       'ardeatino'], dtype=object)

In [15]:
agoda_reduced.head()

,titolo,prezzo,indirizzo,città,numero_recensioni,valutazione
0,hotel valentino,258 €,"terni, terni",Terni,2 recensioni,4.0
1,garden hotel terni,138 €,"terni, terni",Terni,505 recensioni,4.0
2,hotel michelangelo palace spa,174 €,"terni, terni",Terni,718 recensioni,4.0
3,classic hotel tulipano,98 €,"terni, terni",Terni,453 recensioni,3.0
4,de hotel paris,69 €,"terni, terni",Terni,11 recensioni,3.0


In [16]:
booking_reduced.head()

,titolo,prezzo,indirizzo,città,numero_recensioni,valutazione_booking
0,de hotel paris,€ 69,terni,Terni,869 recensioni,NaN
1,bb cospea,€ 55,terni,Terni,674 recensioni,3.0
2,bedbreakfast rosy,€ 77,terni,Terni,330 recensioni,4.0
3,angolo bb verde,€ 69,terni,Terni,219 recensioni,3.0
4,filomena guest house santa,€ 74,terni,Terni,11 recensioni,3.0


Great! Now we have what we need to do our analysis, and not a whole lot of extraneous information. Let's move on to preprocessing.

# pulizia titolo da parole roma e rome

In [17]:
#nuova pulizia indirizzo
agoda_reduced['titolo'] = (
    agoda_reduced['titolo']
    .str.lower()  # tutto minuscolo
    .str.replace(r'\s*-\s*.*$', '', regex=True)  # rimuove tutto dopo il trattino incluso
    .str.replace(r',?\s*roma\b', '', regex=True)  # rimuove "roma" opzionalmente preceduta da virgola
    .str.replace(r',?\s*rome\b', '', regex=True)  # rimuove "centrale" come parola intera
    .str.strip()  # rimuove spazi iniziali/finali
)

/var/folders/j4/9fk3st2d30g22vff83pj6cnw0000gn/T/ipykernel_8489/2607220419.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agoda_reduced['titolo'] = (


In [18]:
#nuova pulizia indirizzo
booking_reduced['titolo'] = (
    booking_reduced['titolo']
    .str.lower()  # tutto minuscolo
    .str.replace(r'\s*-\s*.*$', '', regex=True)  # rimuove tutto dopo il trattino incluso
    .str.replace(r',?\s*roma\b', '', regex=True)  # rimuove "roma" opzionalmente preceduta da virgola
    .str.replace(r',?\s*rome\b', '', regex=True)  # rimuove "centrale" come parola intera
    .str.strip()  # rimuove spazi iniziali/finali
)

/var/folders/j4/9fk3st2d30g22vff83pj6cnw0000gn/T/ipykernel_8489/600013662.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  booking_reduced['titolo'] = (


# pulizia prezzo

In [19]:
# Applica la pulizia
booking_reduced['prezzo'] = pulisci_prezzo(booking_reduced['prezzo'])
agoda_reduced['prezzo'] = pulisci_prezzo(agoda_reduced['prezzo'])

/var/folders/j4/9fk3st2d30g22vff83pj6cnw0000gn/T/ipykernel_8489/2615639504.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  booking_reduced['prezzo'] = pulisci_prezzo(booking_reduced['prezzo'])
/var/folders/j4/9fk3st2d30g22vff83pj6cnw0000gn/T/ipykernel_8489/2615639504.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agoda_reduced['prezzo'] = pulisci_prezzo(agoda_reduced['prezzo'])


# pulizia numero recensioni

In [20]:
# su booking
booking_reduced['numero_recensioni'] = booking_reduced['numero_recensioni']\
    .astype(str)\
    .str.extract(r'(\d+)')\
    .astype(float)

# su agoda
agoda_reduced['numero_recensioni'] = agoda_reduced['numero_recensioni']\
    .astype(str)\
    .str.extract(r'(\d+)')\
    .astype(float)


/var/folders/j4/9fk3st2d30g22vff83pj6cnw0000gn/T/ipykernel_8489/4137780272.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  booking_reduced['numero_recensioni'] = booking_reduced['numero_recensioni']\
/var/folders/j4/9fk3st2d30g22vff83pj6cnw0000gn/T/ipykernel_8489/4137780272.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agoda_reduced['numero_recensioni'] = agoda_reduced['numero_recensioni']\


In [21]:
booking_reduced.head()

,titolo,prezzo,indirizzo,città,numero_recensioni,valutazione_booking
0,de hotel paris,69,terni,Terni,869.0,NaN
1,bb cospea,55,terni,Terni,674.0,3.0
2,bedbreakfast rosy,77,terni,Terni,330.0,4.0
3,angolo bb verde,69,terni,Terni,219.0,3.0
4,filomena guest house santa,74,terni,Terni,11.0,3.0


<a id='Preprocess:-clean-Up-Names-and-separate-to-first-middle-and-last-name'></a>
## Preprocessing the Data
-----

# rinominiamo i dataset

In [22]:
booking_reduced = booking_reduced.rename(columns={'titolo': 'titolo_booking', 'prezzo': 'prezzo_booking', 'indirizzo': 'indirizzo_booking','città':'città_booking','numero_recensioni':'numero_recensioni_booking'})

In [23]:
agoda_reduced = agoda_reduced.rename(columns={'titolo': 'titolo_agoda', 'prezzo': 'prezzo_agoda', 'indirizzo': 'indirizzo_agoda','città':'città_agoda','numero_recensioni':'numero_recensioni_agoda','valutazione':'valutazione_agoda'})

In [24]:
booking_reduced.shape

(886, 6)

# AGGIUNTA FAKE RECORD

-----------------------------------------------------------------

hotel del valentino ma che sta a Roma

In [25]:
# Nuova riga da aggiungere
nuova_riga = pd.DataFrame([{'titolo_agoda': 'hotel valentino', 'prezzo_agoda': 190 ,'indirizzo_agoda':'eur','città_agoda':'Roma'}])

# Aggiunta al dataset
agoda_reduced = pd.concat([agoda_reduced, nuova_riga], ignore_index=True)

In [26]:
# Nuova riga da aggiungere
nuova_riga = pd.DataFrame([{'titolo_agoda': 'paris hotel', 'prezzo_agoda': 110 ,'indirizzo_agoda':'termini','città_agoda':'Roma'}])

# Aggiunta al dataset
agoda_reduced = pd.concat([agoda_reduced, nuova_riga], ignore_index=True)

In [27]:
# Nuova riga da aggiungere
nuova_riga = pd.DataFrame([{'titolo_agoda': 'fiori hotel', 'prezzo_agoda': 81 ,'indirizzo_agoda':'Terni','città_agoda':'Terni'}])

# Aggiunta al dataset
agoda_reduced = pd.concat([agoda_reduced, nuova_riga], ignore_index=True)

inverto ordine parole

(non funziona perchè fa index sulle lettere) cambiare con matching var

errore battitura
classic hotel Tulipano e classic hotel Tulippano sono matchati correttamente

In [28]:
# Nuova riga da aggiungere
nuova_riga = pd.DataFrame([{'titolo_agoda': 'classic hotel tulippano', 'prezzo_agoda': 109 ,'indirizzo_agoda':'Roma','città_agoda':'Roma'}])

# Aggiunta al dataset
agoda_reduced = pd.concat([agoda_reduced, nuova_riga], ignore_index=True)

-----------------------------------------------------------------

# Esportazione dati

In [29]:
agoda_reduced.to_csv(f"{path_data_product}/agoda.csv")

In [30]:
booking_reduced.to_csv(f"{path_data_product}/booking.csv")